In [ ]:
import re, time, random
import requests
from bs4 import BeautifulSoup
import pandas as pd

# -------------------------------------------------------
# 1️⃣ 기본 설정
# -------------------------------------------------------
BASE_M = "https://m.10000recipe.com"
BASE_W = "https://www.10000recipe.com"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# cat4 카테고리 매핑
CAT4 = {
    "밑반찬": 63, "메인반찬": 56, "국/탕": 54, "찌개": 55, "면/만두": 53,
    "밥/죽/떡": 52, "퓨전": 61, "양식": 65, "샐러드": 64, "스프": 68, "기타": 62,
}


# -------------------------------------------------------
# 2️⃣ HTML 요청 함수
# -------------------------------------------------------
def get_soup(url, params=None, max_retry=3, pause=1.5):
    for i in range(max_retry):
        r = requests.get(url, params=params, headers=HEADERS, timeout=15)
        if r.status_code == 200:
            return BeautifulSoup(r.text, "html.parser")
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(pause * (2 ** i) + random.random())
            continue
        raise RuntimeError(f"HTTP {r.status_code} for {url}")
    raise RuntimeError(f"Retry exceeded for {url}")


# -------------------------------------------------------
# 3️⃣ 리스트 페이지 파싱
# -------------------------------------------------------
def parse_list_page(soup):
    cards = []
    for a in soup.select("a[href*='/recipe/']"):
        href = a.get("href", "")
        m = re.search(r"/recipe/(\d+)", href)
        if not m:
            continue
        rid = int(m.group(1))
        title = a.get_text(strip=True)
        card = a.find_parent()
        text = card.get_text(" ", strip=True) if card else title

        author = None
        rating = None
        views = None

        m_views = re.search(r"조회수\s*([\d,]+)", text)
        if m_views:
            views = int(re.sub(r"[^\d]", "", m_views.group(1)))

        m_rating = re.search(r"([0-5]\.?[0-9]?)\s*\(", text)
        if m_rating:
            try:
                rating = float(m_rating.group(1))
            except:
                pass

        m_author = re.search(r"by\s*([^\s]+)", text)
        if m_author:
            author = m_author.group(1)

        cards.append({
            "recipe_id": rid,
            "title": title,
            "author": author,
            "rating": rating,
            "views": views,
            "url": f"{BASE_W}/recipe/{rid}"
        })
    uniq = {c["recipe_id"]: c for c in cards}
    return list(uniq.values())


# -------------------------------------------------------
# 4️⃣ 카테고리별 수집 (필터 없음)
# -------------------------------------------------------
def crawl_list_by_cat4(cat4, start_page=1, end_page=5, order="read"):
    """cat4 카테고리를 페이지네이션으로 훑기 (필터링 없음)"""
    all_cards = []
    for p in range(start_page, end_page + 1):
        params = {"cat4": cat4, "order": order, "page": p}
        soup = get_soup(f"{BASE_M}/recipe/list.html", params=params)
        cards = parse_list_page(soup)
        all_cards.extend(cards)
        print(f"[p={p}] 수집: {len(cards)}개")
        time.sleep(1.0 + random.random())

    uniq = {c["recipe_id"]: c for c in all_cards}
    return list(uniq.values())


# -------------------------------------------------------
# 5️⃣ 상세 페이지 파싱
# -------------------------------------------------------
def parse_detail(recipe_id):
    url = f"{BASE_W}/recipe/{recipe_id}"
    soup = get_soup(url)

    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None
    summary = soup.select_one(".view2_summary_in, .view2_summary")
    summary = summary.get_text("\n", strip=True) if summary else None

    meta_text = soup.get_text(" ", strip=True)
    servings = None; time_text = None; difficulty = None

    m_serv = re.search(r"(\d+\s*인분)", meta_text)
    if m_serv: servings = m_serv.group(1)
    m_time = re.search(r"(\d+\s*분)", meta_text)
    if m_time: time_text = m_time.group(1)
    for diff in ["초급", "중급", "고급"]:
        if diff in meta_text:
            difficulty = diff; break

    ingredients = []
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue
        ingredients.append({"name": text.split()[0], "amount_raw": text})

    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        step_txt = el.get_text(" ", strip=True)
        img = el.select_one("img")
        img_url = img["src"] if img and img["src"].startswith("http") else None
        steps.append({"step_no": i, "text": step_txt, "image_url": img_url})

    return {
        "recipe_id": recipe_id,
        "title": title,
        "summary": summary,
        "servings": servings,
        "time_text": time_text,
        "difficulty": difficulty,
        "ingredients": ingredients,
        "steps": steps,
        "url": url
    }


# -------------------------------------------------------
# 6️⃣ DataFrame 변환 및 저장
# -------------------------------------------------------
def to_dataframes(records):
    rec_rows, ing_rows, step_rows = [], [], []
    for r in records:
        rec_rows.append({
            "recipe_id": r["recipe_id"],
            "title": r["title"],
            "summary": r["summary"],
            "servings": r["servings"],
            "time_text": r["time_text"],
            "difficulty": r["difficulty"],
            "url": r["url"]
        })
        for ing in r["ingredients"]:
            ing_rows.append({"recipe_id": r["recipe_id"], **ing})
        for st in r["steps"]:
            step_rows.append({"recipe_id": r["recipe_id"], **st})
    return (
        pd.DataFrame(rec_rows).drop_duplicates(subset=["recipe_id"]),
        pd.DataFrame(ing_rows),
        pd.DataFrame(step_rows)
    )


# -------------------------------------------------------
# 7️⃣ 실행부
# -------------------------------------------------------
if __name__ == "__main__":
    print("🍳 [RUN] 메인반찬 카테고리 전체 수집 (필터 없음)")

    # 메인반찬 cat4=56 / 페이지 1~3 테스트
    cards = crawl_list_by_cat4(CAT4["메인반찬"], start_page=1, end_page=3, order="read")
    print(f"\n수집된 레시피 수: {len(cards)}개")

    detailed = []
    for c in cards:
        try:
            d = parse_detail(c["recipe_id"])
            detailed.append(d)
            print(f"✔ {d['title']} ({c['url']})")
            time.sleep(1.2 + random.random())
        except Exception as e:
            print("❌ detail error:", c["recipe_id"], e)

    if detailed:
        df_rec, df_ing, df_step = to_dataframes(detailed)
        df_rec.to_csv("recipes_all.csv", index=False)
        df_ing.to_csv("ingredients_all.csv", index=False)
        df_step.to_csv("steps_all.csv", index=False)
        print("\n💾 저장 완료: recipes_all.csv / ingredients_all.csv / steps_all.csv")
    else:
        print("⚠️ 수집된 데이터가 없습니다.")



🍳 [RUN] 메인반찬 카테고리 전체 수집 (필터 없음)
[p=1] 수집: 30개
[p=2] 수집: 30개
[p=3] 수집: 30개

수집된 레시피 수: 90개
✔ None (https://www.10000recipe.com/recipe/7036354)
✔ None (https://www.10000recipe.com/recipe/7009798)
✔ None (https://www.10000recipe.com/recipe/6994321)
✔ None (https://www.10000recipe.com/recipe/7047046)
✔ None (https://www.10000recipe.com/recipe/7053724)
✔ None (https://www.10000recipe.com/recipe/7036748)
✔ None (https://www.10000recipe.com/recipe/7037977)
✔ None (https://www.10000recipe.com/recipe/6999323)
✔ 양념이 쏙쏙베인 부드러운 살이 일품 갈치조림 (https://www.10000recipe.com/recipe/671489)
✔ 정갈한 맛...떡잡채 (https://www.10000recipe.com/recipe/676858)
✔ 고민하지 말자! 둘다 먹으면 되지~ ☞후라이드 & 양념치킨☜ (https://www.10000recipe.com/recipe/683092)
✔ ◈ 집에서 따라해보는 짝퉁 교촌치킨 (https://www.10000recipe.com/recipe/684097)
✔ 알싸한~ 간장 삼치 조림 (https://www.10000recipe.com/recipe/684391)
✔ 보쌈에 조림장만 추가했을 뿐인데 맛난 음식으로 변했네 - 동파육 (https://www.10000recipe.com/recipe/692656)
✔ 닭다리만 먹고 싶고나~~닭다리 양념구이~~ (https://www.10000recipe.com/recipe/708454)
✔ 파프리카 

KeyboardInterrupt: 

In [3]:
import re, time, random
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_W = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}


def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def parse_detail(recipe_id):
    """레시피 상세에서 메뉴명, 재료, 순서, best_tit 가져오기"""
    url = f"{BASE_W}/recipe/{recipe_id}"
    soup = get_soup(url)

    # ① 메뉴명
    title_el = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title_el.get_text(strip=True) if title_el else None

    # ② 재료
    ingredients = [li.get_text(" ", strip=True)
                   for li in soup.select(".ready_ingre3 li, .ingre_list li") if li.get_text(strip=True)]

    # ③ 레시피 순서
    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        text = el.get_text(" ", strip=True)
        img = el.select_one("img")
        img_url = img["src"] if img and img["src"].startswith("http") else None
        steps.append({"step_no": i, "text": text, "image_url": img_url})

    # ④ best_tit
    best_tit_el = soup.select_one("div.best_tit b")
    best_tit = best_tit_el.get_text(strip=True) if best_tit_el else None

    return {
        "recipe_id": recipe_id,
        "title": title,
        "ingredients": ingredients,
        "steps": steps,
        "best_tit": best_tit,
        "url": url
    }


if __name__ == "__main__":
    # 🔸 테스트용 한 페이지만
    recipe_id = 6880100  # 원하는 레시피 ID로 변경 가능
    data = parse_detail(recipe_id)

    # 콘솔에서 형식 확인
    print("\n===== 레시피 정보 =====")
    print("제목:", data["title"])
    print("Best 타이틀:", data["best_tit"])
    print("\n[재료]")
    for ing in data["ingredients"]:
        print(" -", ing)
    print("\n[조리 순서]")
    for step in data["steps"]:
        print(f"{step['step_no']}. {step['text']}")
        if step["image_url"]:
            print("   (이미지)", step["image_url"])

    # CSV 저장
    pd.DataFrame([{
        "recipe_id": data["recipe_id"],
        "title": data["title"],
        "best_tit": data["best_tit"],
        "url": data["url"]
    }]).to_csv("recipe_single.csv", index=False)

    pd.DataFrame([{"recipe_id": data["recipe_id"], "ingredient": ing}
                  for ing in data["ingredients"]]).to_csv("recipe_single_ingredients.csv", index=False)

    pd.DataFrame([{"recipe_id": data["recipe_id"], **s}
                  for s in data["steps"]]).to_csv("recipe_single_steps.csv", index=False)

    print("\n💾 저장 완료: recipe_single.csv / recipe_single_ingredients.csv / recipe_single_steps.csv")



===== 레시피 정보 =====
제목: 매콤한 살라미소시지 마늘종볶음 만들기
Best 타이틀: 재료

[재료]
 - 마늘쫑 2단 구매
 - 살라미 소시지 80g 3개 구매
 - 진간장 3큰술 구매
 - 물엿 2큰술 구매
 - 참기름 1큰술 구매
 - 소금 약간 구매
 - 후추 약간 구매

[조리 순서]
1. 마늘종은 흐르는 물에 씻어서 먹기 좋은 크기로 썰어주세요. 저는 4cm 정도의 길이로 준비했어요.
   (이미지) https://recipe1.ezmember.co.kr/cache/recipe/2018/06/02/c59de12b61b9e617163242d3d452a3441.jpg
2. 끓는 물에 마늘종을 넣고 1분간 삶아주세요. 마늘종을 넣고 나서 끓을 때까지 기다리면 마늘종이 너무 익어서 씹는 맛이 사라진답니다. 딱 1분만 삶아주세요.
   (이미지) https://recipe1.ezmember.co.kr/cache/recipe/2018/06/02/f859ba2ab7d54859e1f814cc9c5cf4e91.jpg
3. 마늘종을 삶을 때 소금을 1작은술 넣어서 삶으면 밑간이 되어서 더 맛있어요.
   (이미지) https://recipe1.ezmember.co.kr/cache/recipe/2018/06/02/85488bebb439475eae4d1cd2da5fcede1.jpg
4. 1분간 삶은 마늘종은 차가운 물에 담궈서 식혀주세요.
   (이미지) https://recipe1.ezmember.co.kr/cache/recipe/2018/06/02/ed96bd4cdffa389501c02ee754f9bea51.jpg
5. 살라미 소시지를 먹기 좋은 크기로 썰어줬어요. 살라미가 없다면 햄을 사용하셔도 좋아요.
   (이미지) https://recipe1.ezmember.co.kr/cache/recipe/2018/06/02/96833e01aeb6b51760be34ffe745dd171.jpg
6. 달궈진 팬에 기름을 두르고 마늘종을 넣어주세요.
   (이미지) 

In [4]:
import re, requests, json
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def parse_recipe(recipe_id):
    url = f"{BASE}/recipe/{recipe_id}"
    soup = get_soup(url)

    # RECIPE_NM_KO (메뉴명)
    title_el = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title_el.get_text(strip=True) if title_el else None

    # SUMRY (#relationGoods > div.best_tit > b:nth-child(1))
    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    # NATION_NM (국가)
    nation_el = soup.select_one("span.cate_nm")
    nation = nation_el.get_text(strip=True) if nation_el else None

    # TY_NM (종류)
    type_el = soup.select_one("div.view_category > a:nth-child(2)")
    type_nm = type_el.get_text(strip=True) if type_el else None

    # COOKING_TIME (#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info)
    time_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info")
    cook_time = time_el.get_text(strip=True).replace("분", "") if time_el else None

    # LEVEL_NM (#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3)
    level_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3")
    level = level_el.get_text(strip=True) if level_el else None

    # IRDNT_CODE (재료 분류 — 현재 None)
    irdnt_code = None

    # INGREDIENT_FULL
    ingredients = [
        li.get_text(" ", strip=True)
        for li in soup.select(".ready_ingre3 li, .ingre_list li")
        if li.get_text(strip=True)
    ]

    # STEP_TEXT / STEP_TIP
    steps, tips = [], []
    for el in soup.select("#stepDiv .media-body, .view_step_cont"):
        text = el.get_text(" ", strip=True)
        steps.append(text)
        tip_el = el.select_one(".step_tip")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "NATION_NM": nation,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "IRDNT_CODE": irdnt_code,
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }


if __name__ == "__main__":
    recipe_id = 6880100  # 테스트용 레시피 ID
    data = parse_recipe(recipe_id)
    df = pd.DataFrame([data])
    df.to_csv("recipe_full_v2.csv", index=False, encoding="utf-8-sig")

    print("\n===== 결과 미리보기 =====")
    for k, v in data.items():
        print(f"{k}: {v}")
    print("\n💾 저장 완료 → recipe_full_v2.csv")



===== 결과 미리보기 =====
RECIPE_ID: 6880100
RECIPE_NM_KO: 매콤한 살라미소시지 마늘종볶음 만들기
SUMRY: 소시지마늘종볶음
NATION_NM: None
TY_NM: None
COOKING_TIME: None
LEVEL_NM: 아무나
IRDNT_CODE: None
INGREDIENT_FULL: ["마늘쫑 2단 구매", "살라미 소시지 80g 3개 구매", "진간장 3큰술 구매", "물엿 2큰술 구매", "참기름 1큰술 구매", "소금 약간 구매", "후추 약간 구매"]
STEP_TEXT: ["마늘종은 흐르는 물에 씻어서 먹기 좋은 크기로 썰어주세요. 저는 4cm 정도의 길이로 준비했어요.", "끓는 물에 마늘종을 넣고 1분간 삶아주세요. 마늘종을 넣고 나서 끓을 때까지 기다리면 마늘종이 너무 익어서 씹는 맛이 사라진답니다. 딱 1분만 삶아주세요.", "마늘종을 삶을 때 소금을 1작은술 넣어서 삶으면 밑간이 되어서 더 맛있어요.", "1분간 삶은 마늘종은 차가운 물에 담궈서 식혀주세요.", "살라미 소시지를 먹기 좋은 크기로 썰어줬어요. 살라미가 없다면 햄을 사용하셔도 좋아요.", "달궈진 팬에 기름을 두르고 마늘종을 넣어주세요.", "마늘종을 뒤집어 주며 살짝 볶아주세요.", "살짝 볶아준 마늘종에 살라미를 넣어주세요.", "진간장, 물엿을 넣어주세요.", "양념이 잘 스며들도록 뒤집어주며 졸여주세요.", "통후추를 조금 뿌려줬습니다. 가루후추를 사용하셔도 좋아요.", "설탕이 들어가면 이렇게 윤기가 흐르지 않아요. 물엿을 사용해야 이렇게 윤기가 흐른답니다.", "양념이 모두 졸여졌다면 마무리로 참기름 1큰술을 넣어주세요. 맛있게 완성된 살라미소시지 마늘종볶음입니다. 청양고추나 매운 고춧가루를 넣을 생각도 했지만 그러면 마늘종볶음 자체가 매워지니까, 매운 살라미를 사용했답니다. 이렇게 만드니 마늘종볶음은 달콤짭조름하고 살라미는 매콤해서 두가지 맛으로 먹는 맛이 있더군요. 또 살라미에서 매운기운이 조금은 나와서 그런지 마늘종

In [7]:
import re, requests, json
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def clean_ingredient(text):
    """'구매' 제거하고 재료명 / 용량 분리"""
    text = text.replace("구매", "").strip()
    # 재료명 + 수량/단위 분리 (공백 기준)
    match = re.match(r"(.+?)\s+([\d\s\/\.]*[가-힣A-Za-z%()]+.*)?$", text)
    if match:
        name = match.group(1).strip()
        amount = match.group(2).strip() if match.group(2) else ""
    else:
        name, amount = text, ""
    return {"INGREDIENT_NM": name, "INGREDIENT_AMT": amount}


def parse_recipe(recipe_id):
    url = f"{BASE}/recipe/{recipe_id}"
    soup = get_soup(url)

    # 기본 정보
    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    nation_el = soup.select_one("span.cate_nm")
    nation = nation_el.get_text(strip=True) if nation_el else None

    type_el = soup.select_one("div.view_category > a:nth-child(2)")
    type_nm = type_el.get_text(strip=True) if type_el else None

    time_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info")
    cook_time = time_el.get_text(strip=True).replace("분", "") if time_el else None

    level_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3")
    level = level_el.get_text(strip=True) if level_el else None

    # IRDNT_CODE (현재 None)
    irdnt_code = None

    # ✅ INGREDIENT_FULL: 재료명/용량 나눔 + '구매' 제거
    ingredients = []
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue
        ing = clean_ingredient(text)
        ingredients.append(ing)

    # STEP_TEXT / STEP_TIP
    steps, tips = [], []
    for el in soup.select("#stepDiv .media-body, .view_step_cont"):
        text = el.get_text(" ", strip=True)
        steps.append(text)
        tip_el = el.select_one(".step_tip")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "NATION_NM": nation,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "IRDNT_CODE": irdnt_code,
        # ingredients를 문자열로 저장 (재료명+용량을 각각 묶어서 리스트 형태)
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }


if __name__ == "__main__":
    recipe_id = 6880100  # 테스트용
    data = parse_recipe(recipe_id)
    df = pd.DataFrame([data])
    df.to_csv("recipe_clean_ingredient.csv", index=False, encoding="utf-8-sig")

    print("\n===== 결과 미리보기 =====")
    print("RECIPE_NM_KO:", data["RECIPE_NM_KO"])
    print("\n[INGREDIENT_FULL]")
    for ing in json.loads(data["INGREDIENT_FULL"]):
        print(f" - {ing['INGREDIENT_NM']}     {ing['INGREDIENT_AMT']}")
    print("\n💾 저장 완료 → recipe_clean_ingredient.csv")



===== 결과 미리보기 =====
RECIPE_NM_KO: 매콤한 살라미소시지 마늘종볶음 만들기

[INGREDIENT_FULL]
 - 마늘쫑     2단
 - 살라미     소시지 80g 3개
 - 진간장     3큰술
 - 물엿     2큰술
 - 참기름     1큰술
 - 소금     약간
 - 후추     약간

💾 저장 완료 → recipe_clean_ingredient.csv


In [ ]:
import re, requests, json
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def clean_ingredient(text):
    """'구매' 제거 후 재료명 / 용량 분리"""
    text = text.replace("구매", "").strip()
    match = re.match(r"(.+?)\s+([\d\s\/\.]*[가-힣A-Za-z%()]+.*)?$", text)
    if match:
        name = match.group(1).strip()
        amount = match.group(2).strip() if match.group(2) else ""
    else:
        name, amount = text, ""
    return name, amount


def parse_recipe(recipe_id):
    url = f"{BASE}/recipe/{recipe_id}"
    soup = get_soup(url)

    # 기본 정보
    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    nation_el = soup.select_one("span.cate_nm")
    nation = nation_el.get_text(strip=True) if nation_el else None

    type_el = soup.select_one("div.view_category > a:nth-child(2)")
    type_nm = type_el.get_text(strip=True) if type_el else None

    # ✅ COOKING_TIME (숫자만 남기기)
    time_el = soup.select_one(
        "#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2"
    )
    cook_time = None
    if time_el:
        raw_time = time_el.get_text(strip=True)
        # 숫자만 추출
        numbers = re.findall(r"\d+", raw_time)
        cook_time = int(numbers[0]) if numbers else None

    level_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3")
    level = level_el.get_text(strip=True) if level_el else None

    irdnt_code = None  # (재료 분류 자리만 유지)

    # ✅ INGREDIENT_FULL — dict 구조 {"마늘쫑":"2단"}
    ingredients = {}
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue
        name, amount = clean_ingredient(text)
        ingredients[name] = amount

    # ✅ STEP_TEXT — 번호 붙여 저장
    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        text = el.get_text(" ", strip=True)
        if text:
            steps.append(f"{i}. {text}")

    # ✅ STEP_TIP — 각 단계별 <div id="stepdescrX"> <p> 내용 긁기
    tips = []
    for i in range(1, len(steps) + 1):
        tip_el = soup.select_one(f"#stepdescr{i} > p")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    return {
    "RECIPE_ID": recipe_id,
    "RECIPE_NM_KO": title,
    "SUMRY": summary,
    "NATION_NM": nation,
    "TY_NM": type_nm,
    "COOKING_TIME": cook_time,
    "LEVEL_NM": level,
    "IRDNT_CODE": irdnt_code,
    "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False),
    "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
    "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }
    




if __name__ == "__main__":
    recipe_id = 6880100  # 테스트용 (원하는 레시피 ID로 교체 가능)
    data = parse_recipe(recipe_id)
    df = pd.DataFrame([data])
    df.to_csv("recipe_with_steptip.csv", index=False, encoding="utf-8-sig")

    print("\n===== 미리보기 =====")
    print("RECIPE_NM_KO:", data["RECIPE_NM_KO"])
    print("\n[INGREDIENT_FULL]")
    for k, v in json.loads(data["INGREDIENT_FULL"]).items():
        print(f" - {k}: {v}")
    print("\n[STEP_TEXT + TIP]")
    step_list = json.loads(data["STEP_TEXT"])
    tip_list = json.loads(data["STEP_TIP"])
    for i in range(len(step_list)):
        print(step_list[i])
        if tip_list[i]:
            print(f"  ⮡ TIP: {tip_list[i]}")
    print("\n💾 저장 완료 → recipe_with_steptip.csv")



===== 미리보기 =====
RECIPE_NM_KO: 매콤한 살라미소시지 마늘종볶음 만들기

[INGREDIENT_FULL]
 - 마늘쫑: 2단
 - 살라미: 소시지 80g 3개
 - 진간장: 3큰술
 - 물엿: 2큰술
 - 참기름: 1큰술
 - 소금: 약간
 - 후추: 약간

[STEP_TEXT + TIP]
1. 마늘종은 흐르는 물에 씻어서 먹기 좋은 크기로 썰어주세요. 저는 4cm 정도의 길이로 준비했어요.
2. 끓는 물에 마늘종을 넣고 1분간 삶아주세요. 마늘종을 넣고 나서 끓을 때까지 기다리면 마늘종이 너무 익어서 씹는 맛이 사라진답니다. 딱 1분만 삶아주세요.
3. 마늘종을 삶을 때 소금을 1작은술 넣어서 삶으면 밑간이 되어서 더 맛있어요.
4. 1분간 삶은 마늘종은 차가운 물에 담궈서 식혀주세요.
5. 살라미 소시지를 먹기 좋은 크기로 썰어줬어요. 살라미가 없다면 햄을 사용하셔도 좋아요.
  ⮡ TIP: 살라미가 없다면 햄을 사용하셔도 좋아요.
6. 달궈진 팬에 기름을 두르고 마늘종을 넣어주세요.
7. 마늘종을 뒤집어 주며 살짝 볶아주세요.
8. 살짝 볶아준 마늘종에 살라미를 넣어주세요.
9. 진간장, 물엿을 넣어주세요.
10. 양념이 잘 스며들도록 뒤집어주며 졸여주세요.
11. 통후추를 조금 뿌려줬습니다. 가루후추를 사용하셔도 좋아요.
12. 설탕이 들어가면 이렇게 윤기가 흐르지 않아요. 물엿을 사용해야 이렇게 윤기가 흐른답니다.
13. 양념이 모두 졸여졌다면 마무리로 참기름 1큰술을 넣어주세요. 맛있게 완성된 살라미소시지 마늘종볶음입니다. 청양고추나 매운 고춧가루를 넣을 생각도 했지만 그러면 마늘종볶음 자체가 매워지니까, 매운 살라미를 사용했답니다. 이렇게 만드니 마늘종볶음은 달콤짭조름하고 살라미는 매콤해서 두가지 맛으로 먹는 맛이 있더군요. 또 살라미에서 매운기운이 조금은 나와서 그런지 마늘종도 약간은 매운 기운이 감돌았답니다.

💾 저장 완료 → recipe_with_steptip.csv


In [13]:
import re, requests, json
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# ✅ 종류별 (cat4) 매핑
CAT4_MAP = {
    63: "밑반찬", 56: "메인반찬", 54: "국/탕", 55: "찌개", 60: "디저트",
    53: "면/만두", 52: "밥/죽/떡", 61: "퓨전", 57: "김치/젓갈/장류",
    58: "양념/소스/잼", 65: "양식", 64: "샐러드", 68: "스프",
    66: "빵", 69: "과자", 59: "차/음료/술", 62: "기타"
}

# ✅ 일반 재료 분류 키워드
INGREDIENT_KEYWORDS = {
    "소고기": "소고기", "쇠고기": "소고기", "양지": "소고기",
    "돼지고기": "돼지고기", "삼겹살": "돼지고기", "목살": "돼지고기",
    "닭": "닭고기", "오리": "닭고기", "육수": "육류",
    "채소": "채소류", "양파": "채소류", "대파": "채소류", "당근": "채소류",
    "감자": "채소류", "마늘": "채소류", "버섯": "버섯류", "표고": "버섯류",
    "팽이": "버섯류", "새우": "해물류", "오징어": "해물류", "조개": "해물류",
    "굴": "해물류", "멸치": "해물류", "참치": "해물류",
    "달걀": "달걀/유제품", "계란": "달걀/유제품", "우유": "달걀/유제품", "치즈": "달걀/유제품",
    "햄": "가공식품류", "소시지": "가공식품류", "베이컨": "가공식품류",
    "쌀": "곡류", "밀가루": "곡류", "빵": "곡류", "콩": "콩/견과류",
    "두부": "콩/견과류", "땅콩": "콩/견과류", "호두": "콩/견과류"
}

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def clean_ingredient(text):
    """'구매' 제거 후 재료명 / 용량 분리"""
    text = text.replace("구매", "").strip()
    match = re.match(r"(.+?)\s+([\d\s\/\.]*[가-힣A-Za-z%()]+.*)?$", text)
    if match:
        name = match.group(1).strip()
        amount = match.group(2).strip() if match.group(2) else ""
    else:
        name, amount = text, ""
    return name, amount

def parse_recipe(recipe_id, cat4=None):
    url = f"{BASE}/recipe/{recipe_id}"
    soup = get_soup(url)

    # ✅ 레시피 기본정보
    title = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title.get_text(strip=True) if title else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    # ✅ 인분 (숫자만)
    serving_el = soup.select_one(
        "#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info1"
    )
    servings = None
    if serving_el:
        raw_serving = serving_el.get_text(strip=True)
        nums = re.findall(r"\d+", raw_serving)
        if len(nums) == 1:
            servings = int(nums[0])
        elif len(nums) >= 2:
            servings = round(sum(map(int, nums)) / len(nums), 1)

    # ✅ 조리시간 (숫자만)
    time_el = soup.select_one(
        "#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2"
    )
    cook_time = None
    if time_el:
        raw_time = time_el.get_text(strip=True)
        nums = re.findall(r"\d+", raw_time)
        cook_time = int(nums[0]) if nums else None

    # ✅ 난이도
    level_el = soup.select_one(
        "#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3"
    )
    level = level_el.get_text(strip=True) if level_el else None

    # ✅ 종류별
    if cat4 and cat4 in CAT4_MAP:
        type_nm = CAT4_MAP[cat4]
    else:
        type_nm = None

    # ✅ 재료 파싱
    ingredients = {}
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text:
            continue
        name, amount = clean_ingredient(text)
        ingredients[name] = amount

    # ✅ IRDNT_CODE 자동 분류 (우선순위: 소고기 > 돼지고기 > 닭고기 > 해물류 > 기타)
    irdnt_code = "기타"
    priority_groups = [
        ("소고기", "소고기"), ("쇠고기", "소고기"), ("양지", "소고기"),
        ("돼지고기", "돼지고기"), ("삼겹살", "돼지고기"), ("목살", "돼지고기"),
        ("닭", "닭고기"), ("오리", "닭고기"),
        ("새우", "해물류"), ("오징어", "해물류"), ("조개", "해물류"),
        ("굴", "해물류"), ("멸치", "해물류"), ("참치", "해물류")
    ]

    # 1️⃣ 우선순위 검사
    for keyword, category in priority_groups:
        if any(keyword in name for name in ingredients.keys()):
            irdnt_code = category
            break

    # 2️⃣ 우선순위에 없으면 일반 분류
    if irdnt_code == "기타":
        for name in ingredients.keys():
            for keyword, category in INGREDIENT_KEYWORDS.items():
                if keyword in name:
                    irdnt_code = category
                    break
            if irdnt_code != "기타":
                break

    # ✅ 조리 단계
    steps = []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        text = el.get_text(" ", strip=True)
        if text:
            steps.append(f"{i}. {text}")

    # ✅ 팁
    tips = []
    for i in range(1, len(steps) + 1):
        tip_el = soup.select_one(f"#stepdescr{i} > p")
        tip = tip_el.get_text(strip=True) if tip_el else ""
        tips.append(tip)

    # ✅ 최종 반환
    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "SERVINGS": servings,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "IRDNT_CODE": irdnt_code,
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False).replace('"', "'"),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }

# ✅ 실행
if __name__ == "__main__":
    recipe_id = 6880100  # 테스트용 (예: 살라미 마늘쫑볶음)
    cat4 = 56            # 메인반찬
    data = parse_recipe(recipe_id, cat4=cat4)

    df = pd.DataFrame([data])
    df.to_csv("recipe_final_v2.csv", index=False, encoding="utf-8-sig")

    print("\n===== 결과 미리보기 =====")
    for k, v in data.items():
        print(f"{k}: {v}")
    print("\n💾 저장 완료 → recipe_final_v2.csv")



===== 결과 미리보기 =====
RECIPE_ID: 6880100
RECIPE_NM_KO: 매콤한 살라미소시지 마늘종볶음 만들기
SUMRY: 소시지마늘종볶음
SERVINGS: 6
TY_NM: 메인반찬
COOKING_TIME: 30
LEVEL_NM: 아무나
IRDNT_CODE: 채소류
INGREDIENT_FULL: {'마늘쫑': '2단', '살라미': '소시지 80g 3개', '진간장': '3큰술', '물엿': '2큰술', '참기름': '1큰술', '소금': '약간', '후추': '약간'}
STEP_TEXT: ["1. 마늘종은 흐르는 물에 씻어서 먹기 좋은 크기로 썰어주세요. 저는 4cm 정도의 길이로 준비했어요.", "2. 끓는 물에 마늘종을 넣고 1분간 삶아주세요. 마늘종을 넣고 나서 끓을 때까지 기다리면 마늘종이 너무 익어서 씹는 맛이 사라진답니다. 딱 1분만 삶아주세요.", "3. 마늘종을 삶을 때 소금을 1작은술 넣어서 삶으면 밑간이 되어서 더 맛있어요.", "4. 1분간 삶은 마늘종은 차가운 물에 담궈서 식혀주세요.", "5. 살라미 소시지를 먹기 좋은 크기로 썰어줬어요. 살라미가 없다면 햄을 사용하셔도 좋아요.", "6. 달궈진 팬에 기름을 두르고 마늘종을 넣어주세요.", "7. 마늘종을 뒤집어 주며 살짝 볶아주세요.", "8. 살짝 볶아준 마늘종에 살라미를 넣어주세요.", "9. 진간장, 물엿을 넣어주세요.", "10. 양념이 잘 스며들도록 뒤집어주며 졸여주세요.", "11. 통후추를 조금 뿌려줬습니다. 가루후추를 사용하셔도 좋아요.", "12. 설탕이 들어가면 이렇게 윤기가 흐르지 않아요. 물엿을 사용해야 이렇게 윤기가 흐른답니다.", "13. 양념이 모두 졸여졌다면 마무리로 참기름 1큰술을 넣어주세요. 맛있게 완성된 살라미소시지 마늘종볶음입니다. 청양고추나 매운 고춧가루를 넣을 생각도 했지만 그러면 마늘종볶음 자체가 매워지니까, 매운 살라미를 사용했답니다. 이렇게 만드니 마늘종볶음은 달콤짭조름하고 살라미는 매콤해서 두가지 맛으로 먹는 맛이

In [ ]:
import re, requests, json, time, random, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path


BASE = "https://www.10000recipe.com"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# ✅ 종류별 (cat4)
CAT4_MAP = {
    63: "밑반찬", 56: "메인반찬", 54: "국/탕", 55: "찌개", 53: "면/만두", 52: "밥/죽/떡", 61: "퓨전", 65: "양식", 64: "샐러드", 68: "스프", 62: "기타"
}

# ✅ 재료 분류 키워드 & 우선순위
INGREDIENT_KEYWORDS = {
    "소고기": "소고기", "쇠고기": "소고기", "양지": "소고기",
    "돼지고기": "돼지고기", "삼겹살": "돼지고기", "목살": "돼지고기",
    "닭": "닭고기", "오리": "닭고기",
    "새우": "해물류", "오징어": "해물류", "조개": "해물류",
    "굴": "해물류", "멸치": "해물류", "참치": "해물류",
    "채소": "채소류", "양파": "채소류", "당근": "채소류",
    "감자": "채소류", "버섯": "버섯류", "두부": "콩/견과류",
}
PRIORITY = [
    ("소고기", "소고기"), ("쇠고기", "소고기"), ("양지", "소고기"),
    ("돼지고기", "돼지고기"), ("삼겹살", "돼지고기"),
    ("닭", "닭고기"), ("오리", "닭고기"),
    ("새우", "해물류"), ("오징어", "해물류"), ("조개", "해물류"),
    ("굴", "해물류"), ("멸치", "해물류"), ("참치", "해물류"),
]

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def clean_ingredient(text):
    text = text.replace("구매", "").strip()
    m = re.match(r"(.+?)\s+([\d\s\/\.]*[가-힣A-Za-z%()]+.*)?$", text)
    if m:
        name = m.group(1).strip()
        amount = m.group(2).strip() if m.group(2) else ""
    else:
        name, amount = text, ""
    return name, amount

def parse_recipe(recipe_id, cat4=None):
    soup = get_soup(f"{BASE}/recipe/{recipe_id}")

    # 기본 정보
    title_el = soup.select_one("div.view2_summary h3, h3.view2_title")
    title = title_el.get_text(strip=True) if title_el else None

    summary_el = soup.select_one("#relationGoods > div.best_tit > b:nth-child(1)")
    summary = summary_el.get_text(strip=True) if summary_el else None

    # SERVINGS (숫자만)
    servings = None
    serv_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info1")
    if serv_el:
        nums = re.findall(r"\d+", serv_el.get_text(strip=True))
        if len(nums) == 1: servings = int(nums[0])
        elif len(nums) >= 2: servings = round(sum(map(int, nums))/len(nums), 1)

    # COOKING_TIME (숫자만)
    cook_time = None
    time_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2")
    if time_el:
        nums = re.findall(r"\d+", time_el.get_text(strip=True))
        cook_time = int(nums[0]) if nums else None

    # LEVEL_NM
    level_el = soup.select_one("#contents_area_full > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3")
    level = level_el.get_text(strip=True) if level_el else None

    # TY_NM
    type_nm = CAT4_MAP.get(cat4, None)

    # INGREDIENT_FULL (dict)
    ingredients = {}
    for li in soup.select(".ready_ingre3 li, .ingre_list li"):
        text = li.get_text(" ", strip=True)
        if not text: continue
        name, amount = clean_ingredient(text)
        ingredients[name] = amount

    # IRDNT_CODE (우선순위 → 일반 매핑)
    irdnt_code = "기타"
    for k, cat in PRIORITY:
        if any(k in nm for nm in ingredients.keys()):
            irdnt_code = cat; break
    if irdnt_code == "기타":
        for nm in ingredients.keys():
            for k, cat in INGREDIENT_KEYWORDS.items():
                if k in nm:
                    irdnt_code = cat; break
            if irdnt_code != "기타": break

    # STEP_TEXT & STEP_TIP
    steps, tips = [], []
    for i, el in enumerate(soup.select("#stepDiv .media-body, .view_step_cont"), 1):
        t = el.get_text(" ", strip=True)
        if t: steps.append(f"{i}. {t}")
        tip_el = soup.select_one(f"#stepdescr{i} > p")
        tips.append(tip_el.get_text(strip=True) if tip_el else "")

    return {
        "RECIPE_ID": recipe_id,
        "RECIPE_NM_KO": title,
        "SUMRY": summary,
        "SERVINGS": servings,
        "TY_NM": type_nm,
        "COOKING_TIME": cook_time,
        "LEVEL_NM": level,
        "IRDNT_CODE": irdnt_code,
        "INGREDIENT_FULL": json.dumps(ingredients, ensure_ascii=False).replace('"', "'"),
        "STEP_TEXT": json.dumps(steps, ensure_ascii=False),
        "STEP_TIP": json.dumps(tips, ensure_ascii=False),
    }

# ✅ 모든 페이지 끝까지, 페이지 완료마다 알림
def get_recipe_ids(cat4, cat_name):
    ids = []
    page = 1
    start_ts = time.time()
    while True:
        url = f"{BASE}/recipe/list.html?cat4={cat4}&order=reco&page={page}"
        soup = get_soup(url)

        # ✅ 구조 변경 반영
        links = soup.select("a.common_sp_link, a.common_sp_thumb, div.common_sp_caption_tit.line2 a")
        if not links:
            print(f"[{cat_name}] 더 이상 페이지 없음 — 총 {len(ids)}개 수집, 경과 {time.time()-start_ts:.1f}s")
            break

        count_before = len(ids)
        for link in links:
            href = link.get("href", "")
            if "/recipe/" in href:
                rid_match = re.findall(r"\d+", href)
                if rid_match:
                    ids.append(int(rid_match[0]))
        # ✅ 여기! 한 페이지 끝날 때마다 알림
        collected_this_page = len(ids) - count_before
        print(f"[{cat_name}] page {page} 완료 — 이번 페이지 {collected_this_page}개, 누적 {len(ids)}개", flush=True)

        page += 1
        time.sleep(random.uniform(1.0, 2.0))
    # 중복 제거
    return list(dict.fromkeys(ids))

if __name__ == "__main__":
    all_rows = []
    crawl_start = time.time()

    print(f"📂 작업 폴더: {Path(os.getcwd()).resolve()}")  # 현재 작업 경로 확인

    for cat4, cat_name in CAT4_MAP.items():
        print(f"\n=== [{cat_name}] 전체 페이지 크롤링 시작 ===", flush=True)
        recipe_ids = get_recipe_ids(cat4, cat_name)
        print(f"[{cat_name}] 레시피 ID {len(recipe_ids)}개 수집 완료", flush=True)

        cat_start = time.time()
        for i, rid in enumerate(recipe_ids, 1):
            try:
                row = parse_recipe(rid, cat4=cat4)
                all_rows.append(row)

                # ✅✅ 여기! 10개마다 즉시 저장 + 경로 출력
                if (len(all_rows) % SAVE_EVERY == 0) or (i == len(recipe_ids)):
                    save_progress(all_rows)

                if i % 20 == 0:
                    print(f"[{cat_name}] 상세 {i}/{len(recipe_ids)} 처리", flush=True)

                time.sleep(random.uniform(1.0, 2.0))
            except Exception as e:
                print(f"[{cat_name}] ❗ {rid} 실패: {e}", flush=True)
                continue

        print(f"[{cat_name}] 상세 크롤링 완료 — {len(recipe_ids)}개 / 소요 {time.time()-cat_start:.1f}s", flush=True)

    # ✅ 최종 저장(원자적 저장)
    df = pd.DataFrame(all_rows)
    tmp_final = FINAL_PATH.with_suffix(".tmp.csv")
    df.to_csv(tmp_final, index=False, encoding="utf-8-sig")
    tmp_final.replace(FINAL_PATH)

    print("\n✅ 전체 카테고리 완료 — 총 {}건, 총 소요 {:.1f}s → {}".format(
        len(df), time.time()-crawl_start, FINAL_PATH.resolve()
    ), flush=True)


📂 작업 폴더: C:\githome\Personalized_healthcare_Project\crawling

=== [밑반찬] 전체 페이지 크롤링 시작 ===


[밑반찬] page 1 완료 — 이번 페이지 40개, 누적 40개
[밑반찬] page 2 완료 — 이번 페이지 40개, 누적 80개
[밑반찬] page 3 완료 — 이번 페이지 40개, 누적 120개
[밑반찬] page 4 완료 — 이번 페이지 40개, 누적 160개
[밑반찬] page 5 완료 — 이번 페이지 40개, 누적 200개
[밑반찬] page 6 완료 — 이번 페이지 40개, 누적 240개
[밑반찬] page 7 완료 — 이번 페이지 40개, 누적 280개
[밑반찬] page 8 완료 — 이번 페이지 40개, 누적 320개
[밑반찬] page 9 완료 — 이번 페이지 40개, 누적 360개
[밑반찬] page 10 완료 — 이번 페이지 40개, 누적 400개
[밑반찬] page 11 완료 — 이번 페이지 40개, 누적 440개
[밑반찬] page 12 완료 — 이번 페이지 40개, 누적 480개
[밑반찬] page 13 완료 — 이번 페이지 40개, 누적 520개
[밑반찬] page 14 완료 — 이번 페이지 40개, 누적 560개
[밑반찬] page 15 완료 — 이번 페이지 40개, 누적 600개
[밑반찬] page 16 완료 — 이번 페이지 40개, 누적 640개
[밑반찬] page 17 완료 — 이번 페이지 40개, 누적 680개
[밑반찬] page 18 완료 — 이번 페이지 40개, 누적 720개
[밑반찬] page 19 완료 — 이번 페이지 40개, 누적 760개
[밑반찬] page 20 완료 — 이번 페이지 40개, 누적 800개
[밑반찬] page 21 완료 — 이번 페이지 40개, 누적 840개
[밑반찬] page 22 완료 — 이번 페이지 40개, 누적 880개
[밑반찬] page 23 완료 — 이번 페이지 40개, 누적 920개
[밑반찬] page 24 완료 — 이번 페이지 40개, 누적 960개
[밑반찬] page 25 완료 — 이번 페이지 40개, 누적 1000개
[밑반찬] page 26 완료 — 이번 페이지 40개, 누적 1

In [1]:
test_cat = 63  # 밑반찬
url = f"https://www.10000recipe.com/recipe/list.html?cat4={test_cat}&order=reco&page=1"
html = requests.get(url, headers=HEADERS).text
print("HTML 길이:", len(html))
print("레시피 링크 수:", len(re.findall(r'/recipe/\d+', html)))


NameError: name 'requests' is not defined

In [2]:
import re
import requests
from bs4 import BeautifulSoup

# 테스트용 카테고리 (밑반찬)
test_cat = 63
url = f"https://www.10000recipe.com/recipe/list.html?cat4={test_cat}&order=reco&page=1"

# User-Agent 헤더 추가 (없으면 차단될 수 있음)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# HTML 요청
html = requests.get(url, headers=HEADERS).text
print("HTML 길이:", len(html))

# BeautifulSoup 파싱 (선택적으로 구조 확인 가능)
soup = BeautifulSoup(html, "html.parser")
print("페이지 제목:", soup.title.get_text(strip=True))

# 레시피 링크 탐색
links = soup.select("div.common_sp_caption a.common_sp_link[href*='/recipe/']")
print("레시피 링크 수:", len(links))

# 정규식으로 숫자 추출 (백업 체크)
ids = re.findall(r"/recipe/(\d+)", html)
print("레시피 ID 수 (정규식 추출):", len(ids))

# 예시로 앞 5개만 출력
print("예시 레시피 ID:", ids[:5])


HTML 길이: 128644
페이지 제목: 요리를 즐겁게~ 만개의레시피
레시피 링크 수: 0
레시피 ID 수 (정규식 추출): 80
예시 레시피 ID: ['6993517', '2022', '6984517', '2022', '7022775']


In [3]:
def get_recipe_ids(cat4, cat_name):
    ids = []
    page = 1
    start_ts = time.time()
    while True:
        url = f"{BASE}/recipe/list.html?cat4={cat4}&order=reco&page={page}"
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            html = r.text
        except Exception as e:
            print(f"[{cat_name}] 페이지 {page} 요청 실패: {e}")
            break

        # ✅ 정규식으로 ID 추출
        found = re.findall(r"/recipe/(\d+)", html)
        found = list(dict.fromkeys(found))  # 중복 제거
        if not found:
            print(f"[{cat_name}] 더 이상 페이지 없음 — 총 {len(ids)}개 수집, 경과 {time.time()-start_ts:.1f}s")
            break

        ids.extend(found)
        print(f"[{cat_name}] page {page} 완료 — 이번 페이지 {len(found)}개, 누적 {len(ids)}개", flush=True)

        page += 1
        time.sleep(random.uniform(1.0, 2.0))

    # 중복 제거 후 반환
    return list(dict.fromkeys(ids))
